In [50]:
import mysql.connector

class DB_Connect:
    def dbConnection(self):
    
        cnx = mysql.connector.connect(user = 'root',password='password123!',host='localhost',database='ridesharing');
        cursor = cnx.cursor();
        return cnx;

    def closeConnection(self,connectionObj):
        connectionObj.close();
    

In [51]:
#select * from neworiginal_data where tpep_pickup_datetime > '2016-01-01 00:00:06' order by tpep_pickup_datetime limit 10;
class RideDetailClass:     
    #time window of 5 minutes
    
    def getFirstRecordPickupTimeStamp(self,connObj):
       query = "SELECT tpep_pickup_datetime FROM neworiginal_data limit 1;"
       cursor = connObj.cursor();
       cursor.execute(query);
       record = cursor.fetchone()
       return record[0]
        
    def getRideDetails(self, connObj,skippedRecOffset):
        
        locationDict = dict() #pickup and dropoffs
        fareAmountDict = dict()
        dropoffTimeDict = dict()
        starttime = self.getFirstRecordPickupTimeStamp(connObj).strftime("%Y-%m-%d %H:%M:%S")
        query = "select * from neworiginal_data where TIMESTAMPDIFF(MINUTE," + "'" + starttime + "'" + ", tpep_pickup_datetime) < 2 order by tpep_pickup_datetime limit " + skippedRecOffset + "," + "11111111111111111111;"
        
        #queryCount = "select count(*) from neworiginal_data where TIMESTAMPDIFF(MINUTE," + "'" + starttime + "'" + ", tpep_pickup_datetime) < 1 order by tpep_pickup_datetime limit " + skippedRecOffset + "," + "11111111111111111111;"
        
        #remove the retrieved data?
        cursor = connObj.cursor();
        cursor.execute(query);
        new_offset = 0 #no. of skipped records in the next iteration
        for row in cursor:
            location = list()
            location.append(row[5])
            location.append(row[6])
            location.append(row[8])
            location.append(row[9])

            locationDict[row[13]] = location
            fareAmountDict[row[13]] = row[12]
            dropoffTimeDict[row[13]] = row[2]
            new_offset = new_offset+1
        
        
        return locationDict,fareAmountDict, dropoffTimeDict,new_offset

In [57]:
from math import radians, cos, sin, asin, sqrt
class CalculateHaversine:
    
    def __init__(self,lon1,lat1,lon2,lat2):
        self.lon1 = lon1
        self.lat1 = lat1
        self.lon2 = lon2
        self.lat2 = lat2


    def haversine(self):
    
        # convert decimal degrees to radians 
        self.lon1, self.lat1, self.lon2, self.lat2 = map(radians, [self.lon1, self.lat1, self.lon2, self.lat2])

        # haversine formula 
        dlon = self.lon2 - self.lon1 
        dlat = self.lat2 - self.lat1 
        a = sin(dlat/2)**2 + cos(self.lat1) * cos(self.lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        #r = 6371 # Radius of earth in kilometers. Use 3956 for miles
        return c * 3963.1676

In [67]:
#location dictionary format {"trip_id": <pickup lon, pickup lat, dropoff lon, dropoff lat>}
class EuclideanDistance:
    
    def __init__(self,minEuclideanDis):
        self.minEuc = minEuclideanDis
    
    def getEuclideanDistanceDict(self,locationDictioary):        
        locationDict2 = locationDictioary.copy()
        euclideanDict = dict()
        listofLocationList = list()
        for key,value in locationDictioary.items():
            lon1 = value[0]
            lat1 = value[1]
            trip_id1 = key
            del locationDict2[key] 
            for key2,value2 in locationDict2.items():
                lon2 = value2[0]
                lat2 = value2[1]
                trip_id2 = key2
                haversineObj = CalculateHaversine(float(lon1),float(lat1),float(lon2),float(lat2))
                eucDist = haversineObj.haversine()
                #################store to db ???
                #check if euclidean distance is less than offset
                if(eucDist < self.minEuc):
                    eucKey = str(trip_id1) + "," + str(trip_id2) 
                    euclideanDict[eucKey] = eucDist
                
        print (len(euclideanDict))
        return euclideanDict
 
#test
euclObj = EuclideanDistance(2)
euclObj.getEuclideanDistanceDict(loc)


6516


{'6,7': 1.86797092304863,
 '6,9': 0.35146874458951644,
 '6,10': 1.8371514955742645,
 '6,11': 1.6301304768756224,
 '6,12': 0.14558141069372166,
 '6,16': 0.11109192631551343,
 '6,18': 0.523547616412041,
 '6,19': 0.2630839217400056,
 '6,20': 1.8903760892282997,
 '6,21': 1.1674774189961021,
 '6,22': 1.1849417360402512,
 '6,23': 1.67029119011689,
 '6,24': 1.677420619634195,
 '6,27': 1.6793263908516822,
 '6,31': 1.694565797781463,
 '6,32': 0.8640536939375424,
 '6,40': 0.6751299755644633,
 '6,43': 1.9132064153568904,
 '6,44': 1.3146203802894842,
 '6,46': 1.8000632712066873,
 '6,48': 1.6244427168006519,
 '6,52': 1.9447745533995788,
 '6,53': 1.3108822373854816,
 '6,57': 1.6321991696856668,
 '6,58': 0.30537041636177914,
 '6,60': 1.0231605034366709,
 '6,61': 0.6457528366193784,
 '6,62': 1.6200211121888288,
 '6,67': 1.6400523271367602,
 '6,68': 1.6694538894106574,
 '6,69': 1.9376973833422386,
 '6,75': 1.1746306131008404,
 '6,76': 1.9885418027734327,
 '6,77': 0.46192219561336245,
 '6,80': 0.8370615

In [53]:
#testing block
dbConnObj = DB_Connect()
rideDetails = RideDetailClass()
con = dbConnObj.dbConnection()

loc, fare, drop, offs = rideDetails.getRideDetails(con,'0')


In [46]:
#testing block
for key,value in loc.items():
    print (key)
    print (value)

1
[-73.99037170410156, 40.73469543457031, -73.98184204101562, 40.73240661621094]
2
[-73.98455047607422, 40.6795654296875, -73.95027160644531, 40.78892517089844]
3
[-73.99346923828125, 40.718990325927734, -73.96224212646484, 40.65733337402344]
4
[-73.96062469482422, 40.78133010864258, -73.97726440429688, 40.758514404296875]
5
[-73.98011779785156, 40.74304962158203, -73.91349029541016, 40.76314163208008]
6
[-73.99405670166016, 40.71998977661133, -73.96636199951172, 40.78987121582031]
7
[-73.97942352294922, 40.74461364746094, -73.99203491210938, 40.753944396972656]
8
[-73.94715118408203, 40.791046142578125, -73.92076873779297, 40.865577697753906]
9
[-73.99834442138672, 40.72389602661133, -73.995849609375, 40.68840026855469]
10
[-74.00614929199219, 40.74491882324219, -73.9937973022461, 40.74143981933594]
11
[-74.00430297851562, 40.74224090576172, -74.00736236572266, 40.70693588256836]
12
[-73.99199676513672, 40.71857833862305, -74.00513458251953, 40.73994445800781]
13
[-73.98516082763672, 